In [1]:
import pandas as pd
import numpy as np
import os
import sys
from pathlib import Path
import plotly

project_root = os.path.abspath("..")

if project_root not in sys.path:
    sys.path.append(project_root)

from preprocessing.video_tracking import read_metrica_event_data
from supportFolder.statical_eventTracking import compute_statistics
from supportFolder.plot_pitch import create_soccer_Pitch
from supportFolder.footBall_amination import FootballAnimator

#Hiển thị toàn bộ thông tin cột dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
matchID = 2500045
df_events, df_formations = read_metrica_event_data(game=matchID, saved=True)

df_events[df_events["eventName"] == "Shot"].head()


____Xử lý dữ liệu trận: 2500045___


,ID,matchID,matchPeriod,eventSec,eventSecEnd,startFrame,endFrame,eventName,subEventName,teamID,Team,posBeforeXMeters,posBeforeYMeters,posAfterXMeters,posAfterYMeters,playerID,playerName,playerPosition,toPlayerID,toPlayerName,homeTeamID,awayTeamID,accurate,Goal,ownGoal
69,552205,2500045,1H,183.777936,184.977936,4594,4624,Shot,Shot,1625,Home,75.6,33.32,0.0,0.0,70083,Danilo,Defender,7910.0,David de Gea,1625,1611,0,0,0
333,552493,2500045,1H,1193.521726,1194.721726,29838,29868,Shot,Shot,1625,Home,75.6,23.80,0.0,0.0,8317,David Silva,Midfielder,7910.0,David de Gea,1625,1611,0,0,0
357,552519,2500045,1H,1272.434663,1274.108950,31810,31852,Shot,Shot,1625,Home,98.7,36.72,0.0,0.0,265673,Bernardo Silva,Midfielder,7910.0,David de Gea,1625,1611,1,0,0
426,552593,2500045,1H,1488.106805,1489.437986,37202,37235,Shot,Shot,1625,Home,96.6,34.00,0.0,0.0,8307,V. Kompany,Defender,7910.0,David de Gea,1625,1611,1,1,1
501,552674,2500045,1H,1801.768231,1804.146736,45044,45103,Shot,Shot,1625,Home,94.5,29.92,0.0,0.0,14808,İ. Gündoğan,Midfielder,7910.0,David de Gea,1625,1611,1,1,1


In [3]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1474 entries, 0 to 1473
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                1474 non-null   int64  
 1   matchID           1474 non-null   int64  
 2   matchPeriod       1474 non-null   object 
 3   eventSec          1474 non-null   float64
 4   eventSecEnd       1474 non-null   float64
 5   startFrame        1474 non-null   int64  
 6   endFrame          1474 non-null   int64  
 7   eventName         1474 non-null   object 
 8   subEventName      1474 non-null   object 
 9   teamID            1474 non-null   int64  
 10  Team              1474 non-null   object 
 11  posBeforeXMeters  1474 non-null   float64
 12  posBeforeYMeters  1474 non-null   float64
 13  posAfterXMeters   1474 non-null   float64
 14  posAfterYMeters   1474 non-null   float64
 15  playerID          1474 non-null   int64  
 16  playerName        1474 non-null   object 


In [4]:
df_game = compute_statistics(df_events, group_col='team', drop_KPIS=["centroid", "totalAccuratePasses"])

df_game.head()

,teamID,nbMatches,totalPasses,shareAccuratePasses,meanPassLength,totalShots,totalGoals,totalDuels
0,1611,1,289,82.35,19.336817,5,8,166
1,1625,1,570,88.07,20.284956,16,7,196


In [5]:
df_startGame = compute_statistics(df_events, group_col='player', drop_KPIS=["totalAccuratePasses"], df_formations=df_formations)

df_startTeam1 = df_startGame[(df_startGame["teamID"] == 1625) & (df_startGame['lineup']==1)].copy()

plot_title = "Away team player position"

pitch_startGame = create_soccer_Pitch(theme="tactical")

pitch_startGame.add_position_plot(df_startTeam1, title=plot_title, colour_kpi="totalPasses")


In [6]:
plot_title = "Away team player position in 90 minutes view"

pitch_startGame90 = create_soccer_Pitch(theme="tactical")

pitch_startGame90.add_position_plot(df_startTeam1, title=plot_title, colour_kpi="totalPasses90")


In [7]:
fig_pass = create_soccer_Pitch(theme="tactical")

df_pass = fig_pass.prepare_pass_(df_events, df_startTeam1, show_top_K_percent=80, view_90_minute=True)

plot_title_pass = "Đường truyền đội nhà trong 90 phút - Manchester City"

fig_pass.add_position_plot(df_startTeam1, df_passes= df_pass, title=plot_title_pass, colour_kpi="totalPasses90")

fig_pass.show()

In [8]:
df_startTeam2 = df_startGame[(df_startGame["teamID"] == 1611) & (df_startGame['lineup']==1)].copy()

fig_pass = create_soccer_Pitch(theme="tactical")

df_pass2 = fig_pass.prepare_pass_(df_events, df_startTeam2, show_top_K_percent=80, view_90_minute=True)

plot_title_pass = "Đường truyền đội khách trong 90 phút"

fig_pass.add_position_plot(df_startTeam2, df_passes= df_pass2, title=plot_title_pass, colour_kpi="totalPasses90")

fig_pass.show()

In [12]:
from supportFolder.statical_eventTracking import build_event_tracking, select_hightlight_events, pair_event_phases
from supportFolder.footBall_amination import FootballAnimator


goal_phases = select_hightlight_events(
    df_events=df_events,
    event_types="LEAD_TO_GOAL",
    preSec=15,
    postSec=2,
    allowed_events=["Pass", "Shot", "Duel"]
)

MIN_FRAMES = 100
valid_phase_idx = 0

for i, phase_df in enumerate(goal_phases):
    print(f"▶ Processing phase {i+1}/{len(goal_phases)}")

    tracking_df = build_event_tracking(
        df_events=phase_df,
        event_ids=phase_df["ID"].tolist()
    )

    if tracking_df.empty:
        continue

    n_frames = tracking_df["frame"].nunique()

    if n_frames < MIN_FRAMES:
        print(f"   ⏭ Skip (only {n_frames} frames)")
        continue

    valid_phase_idx += 1
    print(f"   ✅ Render phase {valid_phase_idx} ({n_frames} frames)")

    animator = FootballAnimator(tracking_df)
    fig = animator.build_animation()
    fig.show()

▶ Processing phase 1/9
   ⏭ Skip (only 50 frames)
▶ Processing phase 2/9
   ⏭ Skip (only 25 frames)
▶ Processing phase 3/9
   ✅ Render phase 1 (150 frames)


▶ Processing phase 4/9
   ⏭ Skip (only 50 frames)
▶ Processing phase 5/9
   ✅ Render phase 2 (225 frames)


▶ Processing phase 6/9
   ⏭ Skip (only 25 frames)
▶ Processing phase 7/9
   ✅ Render phase 3 (200 frames)


▶ Processing phase 8/9
   ⏭ Skip (only 50 frames)
▶ Processing phase 9/9
   ⏭ Skip (only 25 frames)
